<a href="https://colab.research.google.com/github/Sunil-1234/Introduction-to-pyspark-MLlib/blob/main/Pyspark_mlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download dataset from https://www.kaggle.com/datasets/saurabh00007/iriscsv

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 42.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=28cba37b55e9de4b7436748a69d3cff93c7302408ff86e4589a9a3da4f897ca1
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import pyspark

In [5]:
#import sparksession to create new session
from pyspark.sql import SparkSession as ss

In [6]:
#creating new session
spark=ss.builder.appName('practice').getOrCreate()

In [8]:
#loading dataset
data=spark.read.csv('IRIS.csv',header=True,inferSchema=True)

In [9]:
data.show()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|        3.0|         1.4| 

In [10]:
data.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)



Using Linear Regression we will try to predict petal_length using sepal_length and sepal_width

In [11]:
#importing required library for performing ml
from pyspark.ml.feature import VectorAssembler


[sepal_length ,Sepal_width]----> new feature--->independent feature

In [12]:
featureassembler=VectorAssembler(inputCols=["sepal_length","sepal_width"],outputCol="Independent Features")

In [14]:
# Transforming our dataset
output=featureassembler.transform(data)

In [15]:
output.show()

+------------+-----------+------------+-----------+-----------+--------------------+
|sepal_length|sepal_width|petal_length|petal_width|    species|Independent Features|
+------------+-----------+------------+-----------+-----------+--------------------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|           [5.1,3.5]|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|           [4.9,3.0]|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|           [4.7,3.2]|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|           [4.6,3.1]|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|           [5.0,3.6]|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|           [5.4,3.9]|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|           [4.6,3.4]|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|           [5.0,3.4]|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|  

From above output we can see tht new column is added from which we predict the value of petal_length

In [16]:
# Creating new dataframe which contain only dependent and independent columnn
finalized_data=output.select("Independent Features","petal_length")

In [18]:
finalized_data.show()

+--------------------+------------+
|Independent Features|petal_length|
+--------------------+------------+
|           [5.1,3.5]|         1.4|
|           [4.9,3.0]|         1.4|
|           [4.7,3.2]|         1.3|
|           [4.6,3.1]|         1.5|
|           [5.0,3.6]|         1.4|
|           [5.4,3.9]|         1.7|
|           [4.6,3.4]|         1.4|
|           [5.0,3.4]|         1.5|
|           [4.4,2.9]|         1.4|
|           [4.9,3.1]|         1.5|
|           [5.4,3.7]|         1.5|
|           [4.8,3.4]|         1.6|
|           [4.8,3.0]|         1.4|
|           [4.3,3.0]|         1.1|
|           [5.8,4.0]|         1.2|
|           [5.7,4.4]|         1.5|
|           [5.4,3.9]|         1.3|
|           [5.1,3.5]|         1.4|
|           [5.7,3.8]|         1.7|
|           [5.1,3.8]|         1.5|
+--------------------+------------+
only showing top 20 rows



In [19]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
#Defining dependent and independent column
regressor=LinearRegression(featuresCol='Independent Features', labelCol='petal_length')
#fitting value into linear regression
regressor=regressor.fit(train_data)

In [20]:
### Coefficients
regressor.coefficients

DenseVector([1.7158, -1.4327])

In [21]:
### Intercepts
regressor.intercept

-1.954103133941681

In [22]:
### Predict the output value using evaluate method of linear-regression
pred_results=regressor.evaluate(test_data)

In [23]:
#printing prediction value
pred_results.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------------+------------------+
|Independent Features|petal_length|        prediction|
+--------------------+------------+------------------+
|           [4.4,2.9]|         1.4|1.4407730462142554|
|           [4.4,3.2]|         1.3|1.0109641866986065|
|           [4.6,3.4]|         1.4|1.0675929086339553|
|           [4.8,3.0]|         1.4| 1.983839349600603|
|           [4.8,3.0]|         1.4| 1.983839349600603|
|           [4.8,3.4]|         1.6|1.4107608702464047|
|           [4.9,3.0]|         1.4| 2.155423330406829|
|           [5.0,2.0]|         3.5|3.7597035095985483|
|           [5.0,2.3]|         3.3|   3.3298946500829|
|           [5.0,3.0]|         1.6|2.3270073112130527|
|           [5.1,3.8]|         1.5|1.3524343333108801|
|           [5.1,3.8]|         1.6|1.3524343333108801|
|           [5.2,2.7]|         3.9| 3.099984132341149|
|           [5.2,3.4]|         1.4|2.0970967934713025|
|           [5.4,3.4]|         1.7| 2.440264755083752|
|         

In [24]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.5867024229031251, 0.49732075019771743)